In [1]:
import os
import argparse
import logging
logging.basicConfig(level=logging.DEBUG)
from common import find_mxnet, data, fit
from common.util import download_file
from ilab_iterator import ilab_iterator, Multi_ilab_iterator, Single_ilab_iterator, random_task_iterator
import mxnet as mx
import numpy as np
import numpy 

class Cross_Entropy(mx.metric.EvalMetric):
    """Calculate accuracies of multi label"""

    def __init__(self):
        super(Cross_Entropy, self).__init__('cross-entropy')
    def update(self, labels, preds):
        mx.metric.check_label_shapes(labels, preds)
        label = labels[0].asnumpy()
        pred = preds[0].asnumpy()
        for i in range(label.shape[0]):
            prob = pred[i,numpy.int64(label[i])]
            if len(labels) == 1:
                self.sum_metric += (-numpy.log(prob)).sum()
        self.num_inst += label.shape[0]         

        

In [2]:
def places_iterator(data_dir,batch_size):
    train = mx.io.ImageRecordIter(
            path_imgrec         = os.path.join(data_dir, "places365_train_shuf.rec"),
            label_width         = 1,
            data_name           = 'data',
            label_name          = 'softmax1_label',
            data_shape          = (3, 224, 224),
            batch_size          = batch_size,
            pad                 = 0,
            fill_value          = 127,  # only used when pad is valid
            rand_crop           = True,
            max_random_scale    = 1.0,  # 480 with imagnet, 32 with cifar10
            min_random_scale    = 1.0, #0.533,  # 256.0/480.0
            max_aspect_ratio    =  0.25,
            random_h            = 36,  # 0.4*90
            random_s            = 50,  # 0.4*127
            random_l            = 50,  # 0.4*127
            max_rotate_angle    = 10,
            max_shear_ratio     = 0.1,
            rand_mirror         = True,
            shuffle             = True)
            #num_parts           = kv.num_workers,
            #part_index          = kv.rank)
    val = mx.io.ImageRecordIter(
            path_imgrec         = os.path.join(data_dir, "places365_val.rec"),
            label_width         = 1,
            data_name           = 'data',
            label_name          = 'softmax1_label',
            batch_size          = batch_size,
            data_shape          = (3, 224, 224),
            rand_crop           = False,
            rand_mirror         = False)
            #num_parts           = kv.num_workers,
            #part_index          = kv.rank)
    return train, val

In [3]:
#prepare the single_dataset iterators
#ilab
batch_size = 640
data_dir='/efs/datasets/users/furlat/places'
#places
train_places, val_places = places_iterator(data_dir,batch_size)


In [4]:
from importlib import import_module
net = import_module('symbols.resnet_md')
image_shape = '3,224,224'
label_names = [train_places.provide_label[0][0]]
num_classes=[365]
sym = net.get_gated_vmt_symbol(num_classes, 50, image_shape, conv_workspace=256)  
sym=sym[0]

In [5]:
ctx=[mx.gpu(0),mx.gpu(1),mx.gpu(2),mx.gpu(3),mx.gpu(4),mx.gpu(5),mx.gpu(6),mx.gpu(7),mx.gpu(8),mx.gpu(9),mx.gpu(10),mx.gpu(11),mx.gpu(12),mx.gpu(13),mx.gpu(14),mx.gpu(15)]

mod = mx.mod.Module(sym, label_names=label_names, context=ctx)

model_prefix = '/home/ubuntu/results/gated_places_r50'

checkpoint = mx.callback.module_checkpoint(mod,model_prefix)

#lr_schedule it isimilar to the CIFAR100 schedule but half length of the steps
schedule = [240000,480000,720000]
#sym, arg_params, aux_params = \
 #       mx.model.load_checkpoint(model_prefix, 6)
#mod.bind(data_shapes=train_imag.provide_data,
#         label_shapes=val_imag.provide_label)    
#mod.set_params(arg_params, aux_params)

mod.fit(train_places,
        eval_data=val_places,
        eval_metric=[Cross_Entropy()],
        batch_end_callback = mx.callback.log_train_metric(5),
        epoch_end_callback=checkpoint,
        allow_missing=False,
        begin_epoch=0,
        log_prefix = model_prefix,
        optimizer_params={'learning_rate':0.1, 'momentum': 0.9,'wd':0.0001, 'lr_scheduler': mx.lr_scheduler.MultiFactorScheduler(step=schedule,factor=0.1) },
        num_epoch=200)

INFO:root:Iter[0] Batch[0] Train-cross-entropy=5.923199
INFO:root:Iter[0] Batch[5] Train-cross-entropy=6.176596
INFO:root:Iter[0] Batch[10] Train-cross-entropy=6.323463
INFO:root:Iter[0] Batch[15] Train-cross-entropy=6.511440
INFO:root:Iter[0] Batch[20] Train-cross-entropy=6.526879
INFO:root:Iter[0] Batch[25] Train-cross-entropy=6.537637
INFO:root:Iter[0] Batch[30] Train-cross-entropy=6.506954
INFO:root:Iter[0] Batch[35] Train-cross-entropy=6.486938


KeyboardInterrupt: 

In [10]:
dicti=mod.get_params()


({'bn0_beta': <NDArray 64 @cpu(0)>,
  'bn0_gamma': <NDArray 64 @cpu(0)>,
  'bn1_beta': <NDArray 2048 @cpu(0)>,
  'bn1_gamma': <NDArray 2048 @cpu(0)>,
  'bn_data_beta': <NDArray 3 @cpu(0)>,
  'bn_data_gamma': <NDArray 3 @cpu(0)>,
  'conv0_weight': <NDArray 64x3x7x7 @cpu(0)>,
  'fc1_bias': <NDArray 365 @cpu(0)>,
  'fc1_weight': <NDArray 365x2048 @cpu(0)>,
  'stage1_unit1_bn1_beta': <NDArray 64 @cpu(0)>,
  'stage1_unit1_bn1_gamma': <NDArray 64 @cpu(0)>,
  'stage1_unit1_bn2_beta': <NDArray 64 @cpu(0)>,
  'stage1_unit1_bn2_gamma': <NDArray 64 @cpu(0)>,
  'stage1_unit1_bn3_beta': <NDArray 64 @cpu(0)>,
  'stage1_unit1_bn3_gamma': <NDArray 64 @cpu(0)>,
  'stage1_unit1_conv1_weight': <NDArray 64x64x1x1 @cpu(0)>,
  'stage1_unit1_conv2_weight': <NDArray 64x64x3x3 @cpu(0)>,
  'stage1_unit1_conv3_weight': <NDArray 256x64x1x1 @cpu(0)>,
  'stage1_unit1_sc_weight': <NDArray 256x64x1x1 @cpu(0)>,
  'stage1_unit1gate': <NDArray 1 @cpu(0)>,
  'stage1_unit2_bn1_beta': <NDArray 256 @cpu(0)>,
  'stage1_unit2

In [19]:
dicti[0]['stage2_unit4gate'].asnumpy()

array([-0.00254174], dtype=float32)